# Caffe 2 Notebook

This notebook is for using Caffe 2 to train a neural network on Jet Data in the same way that the Keras Notebook in this Github does. We preprocess the data in the same way as the keras notebook, but the way we load this data into the Caffe API will be different and the construction of our CNN will also be different.


In [12]:
import numpy as np
import pandas as pd 
import h5py
import convert_data

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import Normalize, LogNorm

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

np.random.seed(7)

from caffe2.python import core, utils, workspace
from caffe2.proto import caffe2_pb2



In [13]:
images, labels = convert_data.images_and_labels() # This is all we need for preprocessing the data 

## First lines of Caffe 2 

This is where we start with the real heart of this program. First, we have to load in our data in a way that Caffe 2 can interpret it. This is in the form of a prototext file.

The example below is just loading the format of the first image (`image[0]`) and the first label (`label[0]`)

In [14]:
feature_and_label = caffe2_pb2.TensorProtos()
feature_and_label.protos.extend([
    utils.NumpyArrayToCaffe2Tensor(images[0]),
    utils.NumpyArrayToCaffe2Tensor(labels[0])])
print('This is what the tensor proto looks like for a feature and its label:')
print(str(feature_and_label))
print('This is the compact string that gets written into the db:')
print(feature_and_label.SerializeToString())

This is what the tensor proto looks like for a feature and its label:
protos {
  dims: 26
  dims: 32
  dims: 1
  data_type: FLOAT
  float_data: 617.601318359
  float_data: 1025.45581055
  float_data: 0.0
  float_data: 1011.24029541
  float_data: 99.9588394165
  float_data: 333.287322998
  float_data: 1539.86938477
  float_data: 1168.76464844
  float_data: 738.572998047
  float_data: 372.952453613
  float_data: 593.821166992
  float_data: 1474.41308594
  float_data: 486.048217773
  float_data: 149.646026611
  float_data: 920.097412109
  float_data: 287.132598877
  float_data: 971.567626953
  float_data: 858.580322266
  float_data: 16.8235168457
  float_data: 208.776351929
  float_data: 1155.65429688
  float_data: 977.845275879
  float_data: 1117.21203613
  float_data: 206.268203735
  float_data: 791.629821777
  float_data: 169.105651855
  float_data: 1758.08557129
  float_data: 1295.36657715
  float_data: 139.9737854
  float_data: 2443.47216797
  float_data: 1165.4888916
  float_data: 2

In [15]:
train_features, test_features, train_labels, test_labels = train_test_split(images, labels, test_size=0.33, random_state=42)

## Database storage 

After splitting up our data, we have to store it in a database so that we can set up a workspace for training -- or rather two databases for our training and testing.

In [16]:
def write_db(db_type, db_name, features, labels):
    """
        This writes our prototext file to a database 
    """
    db = core.C.create_db(db_type, db_name, core.C.Mode.write)
    transaction = db.new_transaction()
    for i in range(features.shape[0]):
        feature_and_label = caffe2_pb2.TensorProtos()
        feature_and_label.protos.extend([
            utils.NumpyArrayToCaffe2Tensor(features[i]),
            utils.NumpyArrayToCaffe2Tensor(labels[i])])
        transaction.put(
            'train_%03d'.format(i),
            feature_and_label.SerializeToString())
    # Close the transaction, and then close the db.
    del transaction
    del db

write_db("minidb", "jet_data_train.minidb", train_features, train_labels)
write_db("minidb", "jet_data_test.minidb", test_features, test_labels)

In [17]:
net_proto = core.Net("example_reader")
dbreader = net_proto.CreateDB([], "dbreader", db="jet_data_train.minidb", db_type="minidb")
net_proto.TensorProtosDBInput([dbreader], ["X", "Y"], batch_size=16)

print("The net looks like this:")
print(str(net_proto.Proto()))

The net looks like this:
name: "example_reader_1"
op {
  output: "dbreader"
  name: ""
  type: "CreateDB"
  arg {
    name: "db_type"
    s: "minidb"
  }
  arg {
    name: "db"
    s: "jet_data_train.minidb"
  }
}
op {
  input: "dbreader"
  output: "X"
  output: "Y"
  name: ""
  type: "TensorProtosDBInput"
  arg {
    name: "batch_size"
    i: 16
  }
}



## Workspace 

Now we have to create a workspace for training our data. This puts the data in a format such that we are now ready to start training. To do this we load in our prototext file.

Then using this workspace we add the data in the form of blobs -- labelled X for the images and Y for the labels.

In [18]:
workspace.CreateNet(net_proto)

True

In [19]:
# Let's run it to get batches of features.
workspace.RunNet(net_proto.Proto().name)
print("The first batch of features is:")
print(workspace.FetchBlob("X"))
print("The first batch of labels is:")
print(workspace.FetchBlob("Y"))

# Let's run again.
workspace.RunNet(net_proto.Proto().name)
print("The second batch of feature is:")
print(workspace.FetchBlob("X"))
print("The second batch of labels is:")
print(workspace.FetchBlob("Y"))

The first batch of features is:
[[[[ 306.33765  ]
   [ 227.3192   ]
   [ 409.3287   ]
   ...
   [ 150.08578  ]
   [  28.548187 ]
   [ 105.92039  ]]

  [[ 506.5171   ]
   [ 289.7425   ]
   [1136.0721   ]
   ...
   [1356.388    ]
   [ 510.0272   ]
   [ 256.4638   ]]

  [[  22.651337 ]
   [ 578.2915   ]
   [ 806.6289   ]
   ...
   [2184.9097   ]
   [1836.9945   ]
   [ 399.39374  ]]

  ...

  [[ 780.02277  ]
   [ 209.58765  ]
   [ 189.54083  ]
   ...
   [ 718.20715  ]
   [ 855.4674   ]
   [ 138.2732   ]]

  [[ 483.5382   ]
   [ 475.80716  ]
   [-135.80908  ]
   ...
   [ 716.63684  ]
   [ 132.40765  ]
   [ 397.52432  ]]

  [[  54.84404  ]
   [ 714.00226  ]
   [  84.59537  ]
   ...
   [ -39.587357 ]
   [   0.       ]
   [   3.6428757]]]


 [[[ 178.58865  ]
   [ 542.9438   ]
   [ 768.71155  ]
   ...
   [ 167.2033   ]
   [ 544.95447  ]
   [1105.9581   ]]

  [[ 135.28024  ]
   [1065.2627   ]
   [1209.6837   ]
   ...
   [ 616.99335  ]
   [1116.4938   ]
   [2321.6519   ]]

  [[ 538.3586   ]
   [1

## Next

Now we need to train a model on these features